In [1]:
import requests
from requests import Response
import pandas as pd
aqi_url='https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
response:Response=requests.get(aqi_url)
if response.status_code==200:
    print('下載成功')
else:
    print('下載失敗')

#得到python的資料結構(list或dict)
data_base:dict=response.json()

#json的字串格式
data_json=response.text

from pydantic import BaseModel, Field, field_validator
class Site(BaseModel):
    站點:str=Field(alias='sitename')
    縣市:str=Field(alias='county')
    aqi:int=Field(alias='aqi')
    pm25:float=Field(alias='pm2.5_avg')
    時間:str=Field(alias='datacreationdate')
    #解決model_validate_json空字串的問題
    @field_validator('pm25', mode='before')
    @classmethod
    def sapce_zero(cls, value):
        if value=='':
            value=0.0
        return value

class AQI(BaseModel):
    records:list[Site] #records名字要和資料的一樣(json viewer)


aqi=AQI.model_validate_json(data_json)
#source_data=aqi.model_dump()
source_data=aqi.model_dump()['records'] #(aqi.model_dump()的key是records)

df1=pd.DataFrame(source_data)

下載成功
